In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_21411/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_21411/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_20_6,0.999605,0.829536,0.819274,0.996952,0.001643,1.139892,0.646447,0.004420,0.080837,0.040539,1.025268,0.041178,142.821943,284.157116,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_20_2,0.999571,0.829152,0.819746,0.997509,0.001786,1.142463,0.644758,0.003613,0.081812,0.042255,1.027453,0.042921,142.656101,283.991274,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_20_0,0.999485,0.828856,0.819761,0.998213,0.002144,1.144440,0.644705,0.002592,0.073087,0.046300,1.032960,0.047029,142.290430,283.625603,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_19_6,0.999479,0.828736,0.818797,0.996353,0.002169,1.145244,0.648154,0.006925,0.096336,0.046577,1.033356,0.047311,142.266556,283.601729,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_21_5,0.999596,0.828478,0.820318,0.998270,0.001680,1.146965,0.642713,0.007223,0.066079,0.040987,1.025829,0.041632,142.778025,284.113197,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,model_1_3_3,0.963302,0.784323,0.798820,0.966041,0.152755,1.442232,0.719611,0.102500,0.832464,0.390839,3.348655,0.396995,133.757836,275.093009,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
140,model_1_3_0,0.953383,0.776263,0.794442,0.957903,0.194045,1.496131,0.735268,0.127065,0.934838,0.440506,3.983500,0.447443,133.279327,274.614500,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
141,model_1_2_6,0.942984,0.768790,0.783553,0.887170,0.237333,1.546105,0.774219,0.129791,1.096702,0.487168,4.649053,0.494840,132.876586,274.211759,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
142,model_1_2_2,0.899537,0.716011,0.778164,0.864262,0.418182,1.899038,0.793495,0.156142,1.079427,0.646670,7.429659,0.656854,131.743679,273.078851,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
